# Practical 2 

* This is Ron Ramsay's solutions to a practical problem set given within courses STAT2203/STAT7203 at the University of Queensland in Semester 2, 2020.
* A programmatic approach is taken using Python 3.8.
* Related or sub-matters are sometimes rambled on about.

In [1]:
import itertools
from math import factorial
from typing import Iterable

import numpy as np
import pandas as pd

## Question 1

Warren Buffet challenges Bill Gates to a game of dice. Gates picks one of the
following four-sided dice, then Buffet chooses one of the remaining dice.
Whoever rolls the highest number wins. The three dice have the following numbers
on their faces:

    (die A): 12, 10, 3, 1
    (die B): 9, 8, 7, 2
    (die C): 11, 6, 5, 4
  
What is the probability of Buffett winning the game, assuming he chooses his die
to maximise his probability of winning?

##### Solution:

Let's define the problem's objects.

In [2]:
dice = {  # for each die, the set of equilikely outcomes.
    'A': (12, 10, 3, 1),
    'B': (9, 8, 7, 2),
    'C': (11, 6, 5, 4)}

We need to know which dice perform better against others on average.

We define a function that enumerates the sample space for a pair of dice, indicating the winner for each possible permutation of die faces.

In [3]:
def sample_space_first_beats_second(
        A: Iterable, B: Iterable) -> pd.DataFrame:
    """
    Return a table dimensioned |A|x|B|, representing a sample space 
    corresponding to the cartesian product of the sub-outcomes of `A` and `B`.
    Each overall outcome will be in (1,-1, 0) 
    respectively indicating the conditions of (A > B, A < B, A == B),
    i.e. in the same manner as the C-style `cmp` function.
    """
    return pd.DataFrame(
        data={b: [(a > b) - (a < b) for a in A] for b in B}, 
        index=A)

Below are the sample spaces for each pair combination from the set of three dice. 
Each table shows whether an outcome of the first die (each row) beats an outcome of the second die (each column). Values 1, -1, 0 respectively indicate win, lose, draw.

In [4]:
dice_pairs_sample_spaces = {
    pair_keys: sample_space_first_beats_second(
        dice[pair_keys[0]], dice[pair_keys[1]])
    for pair_keys in itertools.combinations(dice.keys(), r=2)}

for k,v in dice_pairs_sample_spaces.items():
    print(f"\n{k}:\n\n{v}")


('A', 'B'):

    9  8  7  2
12  1  1  1  1
10  1  1  1  1
3  -1 -1 -1  1
1  -1 -1 -1 -1

('A', 'C'):

    11  6   5   4 
12   1   1   1   1
10  -1   1   1   1
3   -1  -1  -1  -1
1   -1  -1  -1  -1

('B', 'C'):

   11  6   5   4 
9  -1   1   1   1
8  -1   1   1   1
7  -1   1   1   1
2  -1  -1  -1  -1


Now we can assess which dice are better, and by how much. We can simply count and compare outcomes to achieve this, since each sub-outcome of a die is assumed to be equilikely.

**Probability of winning**: In all of these tables, there is no drawn outcome (of value 0), i.e. all outcomes are binary, i.e. either a win (1) or a loss (-1). Therefore for these dice the probability of winning is the mean number of winning outcomes, i.e. the number of 'ones' divided by the total number of outcomes (16), e.g. for die A versus die B: 9/16 = 0.5625 = 56.25%.

**Profit margin**: If draws were possible outcomes in this set of dice, the 'probability of winning' might not be sufficient to evaluate a die, because there is no distinguishment as to whether a non-win outcome was a draw (which has no cost) or a loss (which does have cost). We can instead use [profit margin](https://en.wikipedia.org/wiki/Profit_margin), which is the expected profit for a trial when 1 unit of currency is wagered on it. From any of the tables above, (and by the equilikely principle) the expected profit is the mean of all the (1, -1, 0) outcomes, e.g. for die A versus die B: (9(1) + 7(-1))/16 = 1/8 = 0.125.

Here are the evaluations:

In [5]:
data_rows = []
for k,v in dice_pairs_sample_spaces.items():
    prob_win = (v.values == 1).mean()
    profit_margin = v.values.mean()
    if profit_margin < 0:
        k = k[::-1]  # Reverse the keys.
        prob_win = 1 - prob_win  # Take the complement.
        profit_margin *= -1  # Take the negative.
    data_rows.append(
        {'dice': k, 'prob(win)': prob_win, 'profit_margin': profit_margin})
df = pd.DataFrame(data=data_rows)
df.set_index('dice', inplace=True, drop=True)
df.sort_index(inplace=True)
df

,prob(win),profit_margin
dice,,
"(A, B)",0.5625,0.125
"(B, C)",0.5625,0.125
"(C, A)",0.5625,0.125


We see a **cyclical situation** whereby 
A beats B, B beats C, and C beats A, all at the same expectation.

Whichever die Bill Gates picks, (since he is nominated as the first player),
Warren Buffet will then choose the die that has a better expectation.

*Answer to problem:* The **probability** of **Buffett winning** the game is **56.25%**.

The interesting thing about this game, given the particular set of dice, is that
the player who is given the first choice of die is actually at a disadvantage!

Sets of dice like this are [non-transitive](https://en.wikipedia.org/wiki/Nontransitive_dice).

In [6]:
# Tidy up to remove programming objects no longer needed.
del dice
del sample_space_first_beats_second
del dice_pairs_sample_spaces
del data_rows
del df

## Question 2
We draw at random 5 numbers from $\left \{1, ..., 100 \right \}$, *with replacement* 
(e.g. drawing number 9 twice is possible). 
What is the probability that exactly 3 numbers are even?

##### Solution:

A single die-roll to produce an even number, as a *Bernoulli trial*, has a consistent probability of 
$\frac{50}{100} = \frac{1}{2}$,
so let's take advantage of the *equilikely principle*.
In doing so, we will consider the ratio of the number of possible outcomes in our event
to that of the complete sample space.

Let $A$ be our event, 
whereby an outcome has an even number for exactly 3 of the 5 trials.

The number of outcomes in our event is:
$|A| = {^{5}\textrm{C}_{3}} = \binom{5}{3} = \frac{5!}{3!(5-3)!}
= \frac{5 \times 4}{2 \times 1}= 10$.

>This is equivalent to the _binomial coeficient_ which counts the number of ways
>in which $r$ objects can be chosen from $n$ possibilities when order doesn't matter.
>In our case, it's the number of ways we can _choose_ exactly 3 of 5 trials to be even number.

For 5 trials, there are $2^5$ potential outcomes, so the cardinality of our sample space is:
$|\Omega| = 32$.

The ratio of the above cardinalities is the probability of our event:
$P(A) = \frac{|A|}{|\Omega|} = \frac{10}{32} = 0.3125$.

In [7]:
def binomial_coefficient(n: int, r: int) -> int: 
    """
    Return the number of ways in which `r` objects can be chosen 
    from `n` possibilities when order doesn't matter.
    """
    return int(factorial(n) / factorial(r) / factorial(n - r))

n = 5
r = 3

event_cardinality = binomial_coefficient(n, r)
omega_cardinality = 2**n

print("Probability of event:", event_cardinality / omega_cardinality)

Probability of event: 0.3125


In [8]:
# (Tidy up to remove objects no longer needed.)
del n, r, event_cardinality, omega_cardinality  #, binomial_coefficient

## Question 3
We draw at random 5 numbers from $\left \{1, ..., 100 \right \}$, *without replacement*. What is the
probability that exactly 3 numbers are even?

##### Solution:

As in the previous question, the number of elemental outcomes in our event remains the same: $|A| = 10$. However, because numbers are not replaced back into the set between draws, the probabilities of odds and evens is affected after each draw. 
For example, although the probability of an even number for the first draw will remain at $\frac{50}{100}$, probabilities for the second draw are dependent on the first, e.g. if the first draw is even, then the probability of even in the second draw is $\frac{49}{99}$, whereas the probability of odd in the second draw is $\frac{50}{99}$. There are a different sequence of such fractions for each of the 10 permutations (of 3 even and 2 odd draws), but it can be shown (if one posessed skill and stamina) that the product of all 5 fractions for any of the 10 permutations is equal to $\frac{50 \times 49 \times 48 \times 50 \times 49}{100 \times 99 \times 98 \times 97 \times 96}$. The answer to question is $10 \frac{50 \times 49 \times 48 \times 50 \times 49}{100 \times 99 \times 98 \times 97 \times 96} \approx 0.31891$.

In [9]:
def question_3_type_problem(n_pool: int, n_draw: int, n_success: int) -> float:
    
    assert n_pool % 2 == 0, "The pool must hold an even number of elements."
    assert n_pool / 2 >= n_draw, "Drawing more than half of the pool is not catered for in this implementation."
    
    n_perms = binomial_coefficient(n_draw, n_success)
    
    numerators = np.array(
        [n_pool / 2 - i for i in range(n_success)] +
        [n_pool / 2 - i for i in range(n_draw - n_success)])
    
    denominators = np.array([n_pool - i for i in range(n_draw)])
    
    return n_perms * np.product(numerators) / np.product(denominators)

print("Answer: Probability:", question_3_type_problem(n_pool=100, n_draw=5, n_success=3))

Answer: Probability: 0.318910757055087


Here is an (completely untested) answer for the same problem but for a reduced pool of $\left \{1, ..., 10 \right \}$:

In [10]:
print("For reduced pool of 10: Probability:", question_3_type_problem(n_pool=10, n_draw=5, n_success=3))

For reduced pool of 10: Probability: 0.3968253968253968


## Question 4
Let $Y$ be a number drawn at random from $\left \{ 1, 2, ..., 60 \right \}$; (all numbers are equally
likely to be drawn).

(a) What is the probability that $Y$ is divisible by 2?

(b) What is the probability that $Y$ is divisible by 2 or 3?

(c) What is the probability that $Y$ is divisible by 2 or 3 or 5?

##### Solution:

(i) A programmatic solution:

In [11]:
def divisible_by(n: int, divisors: Iterable[int]) -> bool:
    "Return whether `n` is divisible by any of `divisors`."
    for e in divisors:
        if n % e == 0:
            return True
    return False

N = 60  # state space cardinality.
omega = set(range(1, N + 1))  # state space.
events_divisors = ({2}, {2, 3}, {2, 3, 5})

print("Probability of a draw being divisible by an element within a given set:")
for divisors in events_divisors:
    divisibles = {e for e in omega if divisible_by(e, divisors)}
    print(divisors, ":", round(len(divisibles) / N, 4))

Probability of a draw being divisible by an element within a given set:
{2} : 0.5
{2, 3} : 0.6667
{2, 3, 5} : 0.7333


(ii) A paper solution:

Let $A_{n_1|n_2|...}$ be the event where an outcome is divisible by at least one element of $\{n_1, n_2, ...\}$; e.g.

- $A_{2} = \{2, 4, 6, ..., 60 \}$
- $A_{3} = \{3, 6, 9, ..., 60 \}$ 
- $A_{5} = \{5, 10, 15, ..., 60 \}$


- $|A_{2}| = 30$
- $|A_{3}| = 20$
- $|A_{5}| = 12$

We want $P(A_{2|3|5})$, where
$A_{2|3|5} = A_{2} \cup A_{3} \cup A_{5}$. 


$$\left|\bigcup_{i=1}^n A_i\right| = \sum_{k=1}^n (-1)^{k+1} \left( \sum_{1 \leqslant i_1 < \cdots < i_k \leqslant n} | A_{i_1} \cap \cdots \cap A_{i_k} | \right)$$

## Question 5
Let $(a_1, ... , a_n)$ be a random permutation of the integers $ \left \{ 1, 2, ..., n \right \}$ with all permuations equally likely.

   (a) What is the probability that $a_1 = 1$?

   (b) What is the probability that $a_1 = 1$ or $a_2 = 2$?

   (c) What is the probability that $a_1 = 1$ or $a_2 = 2$ or $a_3 = 3$?
   
##### Solution:

The number of permutations in the state space is: $|\Omega| = n!$. 


(a) 

If $a_1$ is fixed, then there remains $(n-1)!$ permutations of the remaining $n-1$ elements.

$$P(a_1 = 1) = \frac{(n-1)!}{n!} = \frac{1}{n}$$


(b)

$$P(a_1 = 1 \vee a_2 = 2)$$ 

$$ = P(a_1 = 1) + P(a_2 = 2) - P(a_1 = 1 \wedge a_2 = 2)$$

$$ = \frac{1}{n} + \frac{1}{n} - \frac {\binom{n}{2}}{n^2}$$

$$ = \frac{2}{n} - \frac {n!}{2!n^2(n-2!)} $$



(c)

$$P(a_1 = 1 \vee a_2 = 2 \vee a_3 = 3)$$ 

$$ = P(a_1 = 1) + P(a_2 = 2) + P(a_3 = 3) - 3 \cdot P(a_1 = 1 \wedge a_2 = 2) - P(a_1 = 1 \wedge a_2 = 2 \wedge a_3 = 3)$$

$$ = \frac{3}{n} - 3\frac {\binom{n}{2}}{n^2} - \frac {\binom{n}{3}}{n^3}$$


## Question 6
Testing whether a multivariate polynomial is equal to zero is a standard problem
in computational complexity. We will consider the following simpler setting. Let
$G$ be a polynomial of degree at most $d$ and assume $G$
is not identically zero.
Let $X$ be a number drawn at random (i.e. all numbers are
equally likely to be drawn) from $ \left \{ 1, 2, ..., 100d \right \}$.
Show that $P \left (  G(X) \neq 0 \right) \ge 0.99$.
(Hint: What does the fundamental theorem of algebra say?)

#### _Solution:_
   
*This solution is adapted from the lecturer's.*

We need to prove $P \left(  G(X) \neq 0 \right) \ge 0.99$.
We will accomplish this instead via the complement of the event; i.e. we'll prove
$P \left (  G(X) = 0 \right) < 0.01%$.

The *fundamental theorem of algebra* informs us that a polynomial $G$ of order
$d$ has at most $d$ roots, i.e. values of $x$ such that $G(x) = 0$.

Every trial of our experiment has a sample space of $100d$, whereas at most
only $d$ outcomes exist that
satisfy our event condition of $G(X)=0$.
As that proportion is only $\frac{1}{100}$ of the sample space, we conclude that
$P \left (  G(X) = 0 \right) \le 0.01%$
for any possible trial. 

For completeness, to 
turn the $\le$ sign of this inequality into a $\lt$, we just have to cite
that polynomials of order $d$ do exist that have *less* than $d$ roots.
